### Create a Website to consume the Web Service created and deployed on ACS

This Notebook provides the details of how to develop a Web App to test the Web Service hosted on ACS as created in the previous Notebook. 
<ul>
<li>First, create a Flask Web App that can call the ACS web service</li>
<li>Next, create a Docker Container of the Web App</li>
<li>Finally, deploy the Image using Web App for Linux in Azure <a href = "https://docs.microsoft.com/en-us/azure/app-service-web/app-service-linux-using-custom-docker-image">Tutorial for deploying the docker Image using Webapp for Linux on Azure</a></li>
<ul>


In [22]:
import os
from os import path
import json

In [80]:
!mkdir nerdetection_ddc
!ls nerdetection_ddc

mkdir: cannot create directory ‘nerdetection_ddc’: File exists
app.py	dockerfile  requirements.txt  static  templates


#### Step 1: Create Files that will be used to render the UI

In [ ]:
%%writefile nerdetection_ddc/static/style.css
body {
    padding-top: 5px;
    padding-bottom: 5px;
}

/* Set padding to keep content from hitting the edges */
.body-content {
    padding-left: 5px;
    padding-right: 5px;
}

/* Set width on the form input elements since they are 100% wide by default */
input,
select,
textarea {
    max-width: 1000px;
}

/* styles for validation helpers */
.field-validation-error {
    color: #b94a48;
}

.field-validation-valid {
    display: none;
}

input.input-validation-error {
    border: 1px solid #b94a48;
}

input[type="checkbox"].input-validation-error {
    border: 0 none;
}

.validation-summary-errors {
    color: #b94a48;
}

.validation-summary-valid {
    display: none;
}

In [ ]:
%%writefile nerdetection_ddc/templates/output.html
<!doctype html> 
<html> 
<head> 
<title>First app</title> 
<!--<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}"> -->
</head> 
<body> 
<center>
<h1>Entity Extractor</h1> 
<form method=post action="/output">
<textarea placeholder = "Enter your text here" name = "content" id = "content"  style="width:1000px; height:250px" >{{ val }}</textarea>
<br>
<input type=submit value='See Results' name='submit_btn'>
<h1>Entity Extractor Output</h1> 
<p> Key: <font color="blue">Disease</font>, <font color="lime">Drug/Chemical</font>
<form>
<p align="left">{{ text|safe }}
</form> 
</center>
</body> 
</html>

In [ ]:
%%writefile nerdetection_ddc/templates/first_app.html
<!doctype html> 
<html> 
<head> 
<title>First app</title> 
<!--<link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}"> -->
</head> 
<body> 
<center>
<h1>Entity Extractor</h1> 
<form method=post action="/output">
<textarea placeholder = "Enter your text here" name = "content" id = "content" style="width:1000px; height:250px">
Baricitinib, Methotrexate, or Baricitinib Plus Methotrexate in Patients with Early Rheumatoid Arthritis Who Had Received Limited or No Treatment with Disease-Modifying-Anti-Rheumatic-Drugs (DMARDs): Phase 3 Trial Results.

Keywords: Janus kinase (JAK), methotrexate (MTX) and rheumatoid arthritis (RA) and Clinical research.

In 2 completed phase 3 studies, baricitinib (bari) improved disease activity with a satisfactory safety profile in patients (pts) with moderately-to-severely active RA who were inadequate responders to either conventional synthetic1 or biologic2DMARDs. This abstract reports results from a phase 3 study of bari administered as monotherapy or in combination with methotrexate (MTX) to pts with early active RA who had limited or no prior treatment with DMARDs. MTX monotherapy was the active comparator.
</textarea>
<br>
<input type=submit value='See Results' name='submit_btn'> 
</form>
</center> 
</body> 
</html>

In [ ]:
%%writefile nerdetection_ddc/templates/_formhelpers.html
{% macro render_field(field) %} 
	<dt>{{ field.label }} 
	<dd>{{ field(**kwargs)|safe }} 
	{% if field.errors %} 
		<ul class=errors> 
		{% for error in field.errors %} 
			<li>{{ error }}</li> 
		{% endfor %} 
		</ul> 
	{% endif %} 
	</dd> 
{% endmacro %}

#### Step 2: Create a Flask Web app that will be used to send request to the web service 

In [81]:
%%writefile nerdetection_ddc/app.py
from flask import Flask, render_template, request
from wtforms import Form, validators
import numpy as np
import requests
import json

app = Flask(__name__)

@app.route('/') 
def hello(): 
    form = request.form 
    return render_template('first_app.html', form = form) 

@app.route('/output', methods=['POST']) 
def output(): 
    form = request.form
    if request.method == 'POST':
        content = request.form['content']
        print content
        app_url = 'specify the url of the service here'

        scoring_url = 'http://{}/score'.format(app_url)

        r = requests.get(scoring_url + '?input=' + content)
        print(r.status_code)

        string = r.content.decode('utf-8')
        json_obj = json.loads(string)
        text_annotated = json_obj['result'][0]   
        print text_annotated
        
        return render_template('output.html', text = text_annotated, val = content)
    
    return render_template('first_app.html', form=form)  
   
if __name__ == '__main__': 
    app.run(host='0.0.0.0')

Overwriting nerdetection_ddc/app.py


#### Step 3<b> Add all the requirements for your website here</b>

In [82]:
%%writefile nerdetection_ddc/requirements.txt
Flask
wtforms
requests
numpy

Overwriting nerdetection_ddc/requirements.txt


In [83]:
image_name = "akshaymehra/nerdetection_ddc"
application_path = 'nerdetection_ddc'
docker_file_location = path.join(application_path, 'dockerfile')
print(docker_file_location)

nerdetection_ddc/dockerfile


<b> Create a Custom Image that has all the requirements in it </b>

In [84]:
%%writefile nerdetection_ddc/dockerfile
FROM ubuntu:16.04
MAINTAINER Akshay Mehra <t-akmehr@microsoft.com>

RUN mkdir /code
WORKDIR /code
ADD . /code/

RUN apt-get update && apt-get install -y --no-install-recommends \
        openmpi-bin \
        python \ 
        python-dev \ 
        python-setuptools \
        python-pip && \
        pip install -r /code/requirements.txt

EXPOSE 5000
CMD ["python", "app.py"]

Overwriting nerdetection_ddc/dockerfile


In [85]:
!sudo docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon 12.29 kB
Step 1/8 : FROM ubuntu:16.04
 ---> d355ed3537e9
Step 2/8 : MAINTAINER Akshay Mehra <t-akmehr@microsoft.com>
 ---> Running in 0a1121cdbe7b
 ---> 2eb5fde2b3d0
Removing intermediate container 0a1121cdbe7b
Step 3/8 : RUN mkdir /code
 ---> Running in dc39c54bdbdc
 ---> dc3609ed6ea0
Removing intermediate container dc39c54bdbdc
Step 4/8 : WORKDIR /code
 ---> c376cda534a0
Removing intermediate container 9c2d0bd0b6b6
Step 5/8 : ADD . /code/
 ---> e616c2865a0c
Removing intermediate container a969c4da2327
Step 6/8 : RUN apt-get update && apt-get install -y --no-install-recommends         openmpi-bin         python         python-dev         python-setuptools         python-pip &&         pip install -r /code/requirements.txt
 ---> Running in 619e44bdded9
Get:1 http://archive.ubuntu.com/ubuntu xenial InRelease [247 kB]
Get:2 http://security.ubuntu.com/ubuntu xenial-security InRelease [102 kB]
Get:3 http://archive.ubuntu.com/ubuntu xenial-updates InRe

In [86]:
!sudo docker login -u username -p password

Login Succeeded


In [87]:
!sudo docker tag $image_name "docker.io/akshaymehra/nerdetection_ddc"

In [88]:
!sudo docker push $image_name

The push refers to a repository [docker.io/akshaymehra/nerdetection_ddc]

63f1889b: Preparing 
7d1798db: Preparing 
215facff: Preparing 
c118947e: Preparing 
f951edf5: Preparing 
2a55830d: Preparing 
2c9a24fc: Preparing 
63f1889b: Pushed  251.9 MB/247.3 MBlatest: digest: sha256:186720527ceb6234334fadbe3662c0b1b01f8c50b41fe2b8c08f0f0d0161de01 size: 1984


In [89]:
print('Docker image name {}'.format(image_name)) 

Docker image name akshaymehra/nerdetection_ddc


In [90]:
### Test Locally 
!sudo docker run -it -p 5000:5000 akshaymehra/nerdetection_ddc
## Access the website on port 5000 of your machine eg. http://your_machine_ip:5000/

#### Step 4: <b> Deploy the Docker Imgae on Azure </b>

Follow <a href = "https://docs.microsoft.com/en-us/azure/app-service-web/app-service-linux-using-custom-docker-image#how-to-set-a-custom-docker-image-for-a-web-app"> this </a>to deploy your image on Azure

Follow <a href="https://docs.microsoft.com/en-us/azure/app-service-web/app-service-linux-using-custom-docker-image#how-to-set-the-port-used-by-your-docker-image"> this</a> to set the port of the web app to 5000 as specified in the Docker File